In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_216.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_98.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_137.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_86_1.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_19_1.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_202.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_159.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_217.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_185_1.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_40_1.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_107.mp4
/kaggle/input/Shop DataSet/non shop lifters/videppppsss_37.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_218_1.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_140_1.mp4
/kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_32_1.mp4
/kag

In [14]:
!nvidia-smi
!python -c "import torch; print(torch.version.cuda); print(torch.cuda.get_device_name(0)); print(torch.cuda.get_device_capability(0))"


Mon Nov 10 11:47:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
!pip install protobuf==3.20.3 --force-reinstall --no-cache-dir
!pip install transformers==4.44.2 accelerate==0.33.0 decord==0.6.0 torch torchvision --upgrade --no-cache-dir
!pip uninstall tensorflow -y
!pip install tensorflow-cpu==2.12.0 --no-cache-dir  # optional: avoids conflicts with HF or protobuf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 

In [23]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_curve, roc_auc_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from decord import VideoReader, cpu
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
DATA_DIR = "/kaggle/input/shoplifting-videos-dataset/Shop DataSet"
OUTPUT_DIR = "/kaggle/working"
NUM_FRAMES = 16
FRAME_SIZE = 224
BATCH_SIZE = 2
EPOCHS = 3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [19]:
labels, corrupted = [], []

subdirs = [("shop lifters", 1), ("non shop lifters", 0)]
for folder, label in subdirs:
    folder_path = os.path.join(DATA_DIR, folder)
    if not os.path.exists(folder_path):
        continue
    for video_file in os.listdir(folder_path):
        if video_file.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
            video_path = os.path.join(folder_path, video_file)
            vid = cv2.VideoCapture(video_path)
            if not vid.isOpened():
                corrupted.append(video_path)
            else:
                labels.append([video_path, label])
            vid.release()

df = pd.DataFrame(labels, columns=["video_path", "label"])
df.to_csv(os.path.join(OUTPUT_DIR, "labels.csv"), index=False)
if corrupted:
    print(f"Corrupted videos: {len(corrupted)}")

In [20]:
def check_empty_frames(video_path, threshold=10):
    vid = cv2.VideoCapture(video_path)
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    empty_frames = 0
    for i in range(min(100, frame_count)):
        ret, frame = vid.read()
        if ret and np.std(frame) < threshold:
            empty_frames += 1
    vid.release()
    return empty_frames / min(100, frame_count) > 0.5

df_filtered = []
for i, row in df.iterrows():
    if not check_empty_frames(row["video_path"]):
        df_filtered.append(row)
df = pd.DataFrame(df_filtered)
df.to_csv(os.path.join(OUTPUT_DIR, "labels_final.csv"), index=False)
print(f"Videos after removing empty frames: {len(df)}")

Videos after removing empty frames: 0


In [33]:
min_count = min(df["label"].value_counts())
df_balanced = pd.concat([
    df[df["label"]==1].sample(min_count, random_state=42),
    df[df["label"]==0].sample(min_count, random_state=42)
])
df_balanced.to_csv(os.path.join(OUTPUT_DIR, "labels_balanced.csv"), index=False)
print(f"Balanced dataset size: {len(df_balanced)}")

KeyError: 'label'

In [42]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)


/kaggle/input
/kaggle/input/Shop DataSet
/kaggle/input/Shop DataSet/non shop lifters
/kaggle/input/Shop DataSet/shop lifters


In [43]:
import os
import pandas as pd


In [44]:
base_dir = "/kaggle/input/Shop DataSet"

shoplifters_dir = os.path.join(base_dir, "shop lifters")
nonshoplifters_dir = os.path.join(base_dir, "non shop lifters")

print("Shoplifters folder exists:", os.path.exists(shoplifters_dir))
print("Non-shoplifters folder exists:", os.path.exists(nonshoplifters_dir))


Shoplifters folder exists: True
Non-shoplifters folder exists: True


In [45]:
shop_videos = [os.path.join(shoplifters_dir, f) for f in os.listdir(shoplifters_dir)]
nonshop_videos = [os.path.join(nonshoplifters_dir, f) for f in os.listdir(nonshoplifters_dir)]

print(f"Shoplifters videos: {len(shop_videos)}")
print(f"Non-shoplifters videos: {len(nonshop_videos)}")


Shoplifters videos: 324
Non-shoplifters videos: 531


In [46]:
df = pd.DataFrame({
    "path": shop_videos + nonshop_videos,
    "label": [1] * len(shop_videos) + [0] * len(nonshop_videos)
})

print(" DataFrame created successfully!")
print("Shape:", df.shape)
df.head()


 DataFrame created successfully!
Shape: (855, 2)


,path,label
0,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1
1,/kaggle/input/Shop DataSet/shop lifters/videyy...,1
2,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1
3,/kaggle/input/Shop DataSet/shop lifters/videyy...,1
4,/kaggle/input/Shop DataSet/shop lifters/videpp...,1


In [47]:
min_count = df["label"].value_counts().min()

df_balanced = pd.concat([
    df[df["label"] == 1].sample(min_count, random_state=42),
    df[df["label"] == 0].sample(min_count, random_state=42)
])

print(" Balanced dataset size:", len(df_balanced))
df_balanced.head()


 Balanced dataset size: 648


,path,label
132,/kaggle/input/Shop DataSet/shop lifters/videyy...,1
108,/kaggle/input/Shop DataSet/shop lifters/videyy...,1
137,/kaggle/input/Shop DataSet/shop lifters/videpp...,1
9,/kaggle/input/Shop DataSet/shop lifters/videpp...,1
180,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1


In [48]:
output_path = "/kaggle/working/labels_balanced.csv"
df_balanced.to_csv(output_path, index=False)

print(" Saved balanced dataset to:", output_path)


 Saved balanced dataset to: /kaggle/working/labels_balanced.csv


In [49]:
df_check = pd.read_csv(output_path)
print(" Reloaded balanced CSV successfully!")
print("Shape:", df_check.shape)
df_check.head()


 Reloaded balanced CSV successfully!
Shape: (648, 2)


,path,label
0,/kaggle/input/Shop DataSet/shop lifters/videyy...,1
1,/kaggle/input/Shop DataSet/shop lifters/videyy...,1
2,/kaggle/input/Shop DataSet/shop lifters/videpp...,1
3,/kaggle/input/Shop DataSet/shop lifters/videpp...,1
4,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1


In [50]:
# Cell 1
import os
import random
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
import cv2
from tqdm.auto import tqdm

# Paths (adjust if you moved dataset)
BASE_DIR = "/kaggle/input/Shop DataSet"
SHOP_DIR = os.path.join(BASE_DIR, "shop lifters")
NONSHOP_DIR = os.path.join(BASE_DIR, "non shop lifters")
OUTPUT_CSV = "/kaggle/working/labels_all.csv"

print("Exists:", os.path.exists(BASE_DIR))
print("Shop lifters folder exists:", os.path.exists(SHOP_DIR))
print("Non-shop lifters folder exists:", os.path.exists(NONSHOP_DIR))


Exists: True
Shop lifters folder exists: True
Non-shop lifters folder exists: True


In [51]:
# Cell 2
# Build CSV of file paths + labels (1 = shoplifter, 0 = non-shoplifter)
shop_files = sorted([os.path.join(SHOP_DIR, f) for f in os.listdir(SHOP_DIR) if not f.startswith('.')])
nonshop_files = sorted([os.path.join(NONSHOP_DIR, f) for f in os.listdir(NONSHOP_DIR) if not f.startswith('.')])

print("Found shop files:", len(shop_files))
print("Found non-shop files:", len(nonshop_files))

df = pd.DataFrame({
    "path": shop_files + nonshop_files,
    "label": [1] * len(shop_files) + [0] * len(nonshop_files)
})

df.to_csv(OUTPUT_CSV, index=False)
print("Saved CSV to:", OUTPUT_CSV)
df.head()


Found shop files: 324
Found non-shop files: 531
Saved CSV to: /kaggle/working/labels_all.csv


,path,label
0,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1
1,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1
2,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1
3,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1
4,/kaggle/input/Shop DataSet/shop lifters/shop_l...,1


In [52]:
# Cell 3
df = pd.read_csv(OUTPUT_CSV)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

train_csv = "/kaggle/working/train_labels.csv"
val_csv = "/kaggle/working/val_labels.csv"
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)

print("Train size:", len(train_df), "Val size:", len(val_df))
print("Train saved to:", train_csv)
print("Val saved to:", val_csv)


Train size: 684 Val size: 171
Train saved to: /kaggle/working/train_labels.csv
Val saved to: /kaggle/working/val_labels.csv


In [53]:
# Cell 4
class VideoFramesDataset(Dataset):
    """
    Reads a video file, samples `num_frames` frames uniformly,
    applies transform to each frame, returns tensor: (num_frames, C, H, W)
    """
    def __init__(self, csv_file, num_frames=8, transform=None, return_path=False):
        self.df = pd.read_csv(csv_file)
        self.num_frames = num_frames
        self.transform = transform
        self.return_path = return_path

    def __len__(self):
        return len(self.df)

    def _read_video_frames(self, video_path, num_frames):
        # Use OpenCV to read frames
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            # attempt to read direct path fallback
            raise RuntimeError(f"Could not open video: {video_path}")

        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total <= 0:
            # fallback: read until end to count
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()
            total = len(frames)
            if total == 0:
                raise RuntimeError(f"No frames in video: {video_path}")
            # sample indexes
            idxs = np.linspace(0, total - 1, num_frames).astype(int)
            sampled = [frames[i] for i in idxs]
            return sampled

        # choose uniform frame indices
        idxs = np.linspace(0, max(total - 1, 0), num_frames).astype(int)
        frames = []
        for i in range(total):
            ret, frame = cap.read()
            if not ret:
                break
            if i in idxs:
                frames.append(frame)
                if len(frames) == num_frames:
                    break
        cap.release()
        # if fewer frames captured (very short video), repeat last frame
        if len(frames) < num_frames:
            if len(frames) == 0:
                raise RuntimeError(f"No frames captured from: {video_path}")
            while len(frames) < num_frames:
                frames.append(frames[-1].copy())
        return frames

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = row["path"]
        label = int(row["label"])

        frames = self._read_video_frames(path, self.num_frames)  # list of BGR frames (H,W,C)

        processed = []
        for f in frames:
            # convert BGR -> RGB
            f = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
            if self.transform:
                f = self.transform(image=f)["image"] if hasattr(self.transform, '__call__') and isinstance(self.transform, dict) else self.transform(f)
            processed.append(f)

        # processed frames are expected to be PIL or tensor; let's ensure tensors (C,H,W)
        # If transform returns numpy array HWC, convert to torch tensor
        frames_tensor = torch.stack([torch.tensor(np.transpose(p, (2,0,1)), dtype=torch.float32) if isinstance(p, np.ndarray) else p for p in processed])
        # normalize to [0,1] if necessary (some transforms already do)
        if frames_tensor.max() > 5.0:  # likely 0-255
            frames_tensor = frames_tensor / 255.0

        if self.return_path:
            return frames_tensor, label, path
        return frames_tensor, label


In [54]:
# Cell 5
# Basic transforms: resize + center crop + normalization (for imagenet pretrained)
IMG_SIZE = 224
num_frames = 8
batch_size = 8  # adjust to your GPU limits

# Using torchvision transforms on PIL images is messy with cv2 arrays: we will use simple manual transform
import torchvision.transforms.functional as TF

def simple_frame_transform(frame_rgb):
    # frame_rgb is numpy array HWC
    img = cv2.resize(frame_rgb, (IMG_SIZE, IMG_SIZE))
    img = img.astype(np.float32) / 255.0
    # convert to tensor C,H,W
    tensor = torch.tensor(np.transpose(img, (2,0,1)), dtype=torch.float32)
    # normalize using ImageNet mean/std
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    tensor = (tensor - mean) / std
    return tensor

# wrap transform so VideoFramesDataset uses it
def transform_wrapper(frame_rgb):
    return simple_frame_transform(frame_rgb).numpy()  # convert to numpy so dataset will convert back to tensor

# Create datasets
train_dataset = VideoFramesDataset(train_csv, num_frames=num_frames, transform=lambda f: simple_frame_transform(f), return_path=False)
val_dataset = VideoFramesDataset(val_csv, num_frames=num_frames, transform=lambda f: simple_frame_transform(f), return_path=False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print("Train batches:", len(train_loader), "Val batches:", len(val_loader))


Train batches: 86 Val batches: 22


In [55]:
# Cell 6
class FrameAggregationModel(nn.Module):
    def __init__(self, backbone_name="resnet50", pretrained=True, num_classes=2, dropout=0.5):
        super().__init__()
        # load resnet50 backbone
        backbone = models.resnet50(pretrained=pretrained)
        # remove classifier layer (fc)
        self.feature_extractor = nn.Sequential(*list(backbone.children())[:-1])  # outputs (B,2048,1,1)
        self.feature_dim = backbone.fc.in_features  # 2048 for resnet50
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: (B, num_frames, C, H, W)
        B, F, C, H, W = x.shape
        x = x.view(B * F, C, H, W)
        feats = self.feature_extractor(x)  # (B*F, feat_dim, 1,1)
        feats = feats.view(B, F, self.feature_dim)  # (B, F, feat_dim)
        # average pooling across frames
        feats = feats.mean(dim=1)  # (B, feat_dim)
        out = self.classifier(feats)  # (B, num_classes)
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FrameAggregationModel(pretrained=True, num_classes=2).to(device)
print(model)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 212MB/s]


FrameAggregationModel(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
  

In [56]:
# Cell 7
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.5)

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch in tqdm(loader):
        inputs, labels = batch  # inputs: (B, F, C, H, W)
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(loader):
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total


In [58]:
# Cell 8
n_epochs = 8
best_val_acc = 0.0
checkpoint_path = "/kaggle/working/best_model.pth"

for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    print(f" Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}")
    print(f" Val   loss: {val_loss:.4f} | Val   acc: {val_acc:.4f}")
    # save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epoch": epoch,
            "val_acc": val_acc
        }, checkpoint_path)
        print(" Saved best model:", checkpoint_path)
    scheduler.step()

print("Training finished. Best val acc:", best_val_acc)


Epoch 1/8


  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

 Train loss: 0.2301 | Train acc: 0.9167
 Val   loss: 0.1251 | Val   acc: 0.9415
 Saved best model: /kaggle/working/best_model.pth
Epoch 2/8


  0%|          | 0/86 [00:00<?, ?it/s]

...

  0%|          | 0/22 [00:00<?, ?it/s]

 Train loss: 0.0360 | Train acc: 0.9839
 Val   loss: 0.0023 | Val   acc: 1.0000
Epoch 8/8


  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

 Train loss: 0.0132 | Train acc: 0.9942
 Val   loss: 0.0005 | Val   acc: 1.0000
Training finished. Best val acc: 1.0


In [59]:
# Cell 9
checkpoint = torch.load("/kaggle/working/best_model.pth", map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
print("Loaded checkpoint from epoch", checkpoint["epoch"], "val_acc", checkpoint["val_acc"])


Loaded checkpoint from epoch 3 val_acc 1.0


In [60]:
# Cell 10
import torch.nn.functional as F

def predict_video(model, video_path, num_frames=8, device=device):
    model.eval()
    ds = VideoFramesDataset(csv_file=None, num_frames=num_frames, transform=lambda f: simple_frame_transform(f), return_path=False)
    # We will use the dataset internal frame reader directly to avoid CSV read
    # copy of logic: read frames and transform
    from pathlib import Path
    frames = ds._read_video_frames(video_path, num_frames)
    proc = [simple_frame_transform(cv2.cvtColor(f, cv2.COLOR_BGR2RGB)) for f in frames]  # each is tensor C,H,W
    x = torch.stack(proc).unsqueeze(0)  # (1, F, C, H, W)
    x = x.to(device)
    with torch.no_grad():
        out = model(x)
        probs = F.softmax(out, dim=1).cpu().numpy()[0]
    return {"prob_nonshop": float(probs[0]), "prob_shop": float(probs[1]), "pred": int(probs[1] > probs[0])}

# quick test on a validation example
sample_path = val_df.iloc[0]["path"]
print("Sample video:", sample_path)
print("Prediction:", predict_video(model, sample_path, num_frames=num_frames))


Sample video: /kaggle/input/Shop DataSet/non shop lifters/shop_lifter_n_122.mp4


ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [61]:
# Cell 11
# NOTE: In Kaggle notebooks, the Gradio app may not open in the notebook output panel.
# You can run this locally on your machine (after downloading the model and dataset) or try in Kaggle with `allow_flagging=False`.
try:
    import gradio as gr
except Exception:
    !pip install -q gradio && python -m pip install -q gradio
    import gradio as gr

def gradio_infer(video_file):
    # video_file is a local path to uploaded video (temp)
    res = predict_video(model, video_file.name if hasattr(video_file, 'name') else video_file, num_frames=num_frames)
    label = "shoplifter" if res["pred"] == 1 else "non-shoplifter"
    return f"Pred: {label} | prob_shop={res['prob_shop']:.3f} prob_nonshop={res['prob_nonshop']:.3f}"

iface = gr.Interface(fn=gradio_infer, inputs=gr.Video(label="Upload video"), outputs="text",
                     title="Shoplifting Detector (simple demo)",
                     description="Uploads a short video; model samples frames and predicts.")
# To run in notebook (it will try to open a server). In Kaggle this may not display; run locally instead.
iface.launch(share=False)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [62]:
!pip install -q transformers accelerate timm gradio opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 27.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.


In [63]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import VideoMAEForVideoClassification, VideoMAEImageProcessor
import torch.nn.functional as F
import gradio as gr

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


Using device: cuda


In [64]:
BASE_DIR = "/kaggle/input/Shop DataSet"
SHOP_DIR = os.path.join(BASE_DIR, "shop lifters")
NONSHOP_DIR = os.path.join(BASE_DIR, "non shop lifters")

shop_files = [os.path.join(SHOP_DIR, f) for f in os.listdir(SHOP_DIR) if f.endswith((".mp4", ".avi"))]
nonshop_files = [os.path.join(NONSHOP_DIR, f) for f in os.listdir(NONSHOP_DIR) if f.endswith((".mp4", ".avi"))]

df = pd.DataFrame({
    "path": shop_files + nonshop_files,
    "label": [1]*len(shop_files) + [0]*len(nonshop_files)
})
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

train_csv = "/kaggle/working/train.csv"
val_csv = "/kaggle/working/val.csv"
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)
print("Train:", len(train_df), "Val:", len(val_df))


Train: 684 Val: 171


In [65]:
class VideoMAEDataset(Dataset):
    def __init__(self, df, processor, num_frames=16, max_len=16):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.num_frames = num_frames
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def _read_video_frames(self, path, num_frames):
        cap = cv2.VideoCapture(path)
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total <= 0:
            raise RuntimeError(f"Empty video: {path}")
        idxs = np.linspace(0, total - 1, num_frames).astype(int)
        frames = []
        for i in range(total):
            ret, frame = cap.read()
            if not ret:
                break
            if i in idxs:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
        cap.release()
        return frames

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        video_path = row["path"]
        label = int(row["label"])
        frames = self._read_video_frames(video_path, self.num_frames)
        processed = self.processor(frames, return_tensors="pt")
        pixel_values = processed["pixel_values"].squeeze(0)  # (num_frames, 3, H, W)
        return {"pixel_values": pixel_values, "labels": torch.tensor(label)}


In [66]:
model_name = "MCG-NJU/videomae-base"
processor = VideoMAEImageProcessor.from_pretrained(model_name)
model = VideoMAEForVideoClassification.from_pretrained(model_name, num_labels=2)
model.to(DEVICE)

train_dataset = VideoMAEDataset(train_df, processor, num_frames=16)
val_dataset   = VideoMAEDataset(val_df, processor, num_frames=16)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/377M [00:00<?, ?B/s]

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
epochs = 3
best_val_acc = 0.0
save_path = "/kaggle/working/videomae_best.pth"

for epoch in range(1, epochs+1):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for batch in tqdm(train_loader):
        pixel_values = batch["pixel_values"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        outputs = model(pixel_values, labels=labels)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * labels.size(0)
        preds = outputs.logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    train_acc = correct / total
    train_loss = total_loss / total

    # Validation
    model.eval()
    val_correct, val_total, val_loss = 0, 0, 0
    with torch.no_grad():
        for batch in val_loader:
            pixel_values = batch["pixel_values"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)
            outputs = model(pixel_values, labels=labels)
            val_loss += outputs.loss.item() * labels.size(0)
            preds = outputs.logits.argmax(dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total
    val_loss = val_loss / val_total

    print(f"Epoch {epoch}: Train loss={train_loss:.4f} acc={train_acc:.4f} | Val loss={val_loss:.4f} acc={val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print("✅ Saved best model:", save_path)


  0%|          | 0/342 [00:00<?, ?it/s]

Epoch 1: Train loss=0.3758 acc=0.8450 | Val loss=0.1403 acc=0.9298
✅ Saved best model: /kaggle/working/videomae_best.pth


  0%|          | 0/342 [00:00<?, ?it/s]

Epoch 2: Train loss=0.1208 acc=0.9532 | Val loss=0.0191 acc=1.0000
✅ Saved best model: /kaggle/working/videomae_best.pth


  0%|          | 0/342 [00:00<?, ?it/s]

Epoch 3: Train loss=0.0473 acc=0.9854 | Val loss=0.1244 acc=0.9532


In [69]:
def predict_video(video_path):
    model.eval()
    frames = VideoMAEDataset(train_df, processor)._read_video_frames(video_path, 16)
    inputs = processor(frames, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
    pred = int(np.argmax(probs))
    return {"prob_nonshop": float(probs[0]), "prob_shop": float(probs[1]), "pred": pred}


In [71]:
def gradio_predict(video_file):
    result = predict_video(video_file.name)
    label = "🛒 Shoplifter" if result["pred"] == 1 else "✅ Non-Shoplifter"
    return f"{label}\nP(shoplifter)={result['prob_shop']:.3f}\nP(non-shop)={result['prob_nonshop']:.3f}"

ui = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Video(label="Upload a short video"),
    outputs="text",
    title="🎥 Shoplifting Detection with VideoMAE",
    description="This fine-tuned VideoMAE model classifies short CCTV videos as shoplifter or non-shoplifter."
)

ui.launch(share=True)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://2d9ad274a094eb52b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
def check_video_properties(video_path, min_frames=NUM_FRAMES, min_resolution=(FRAME_SIZE, FRAME_SIZE)):
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        return False, None
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    vid.release()
    if frame_count < min_frames or width < min_resolution[0] or height < min_resolution[1]:
        return False, (frame_count, width, height)
    return True, (frame_count, width, height)

df_cleaned = []
for i, row in df.iterrows():
    valid, stats = check_video_properties(row["video_path"])
    if valid:
        df_cleaned.append(row)
df = pd.DataFrame(df_cleaned)
df.to_csv(os.path.join(OUTPUT_DIR, "labels_cleaned.csv"), index=False)
print(f"Videos after resolution/frame check: {len(df)}")

Videos after resolution/frame check: 0


In [29]:
def check_empty_frames(video_path, threshold=10):
    vid = cv2.VideoCapture(video_path)
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    empty_frames = 0
    for i in range(min(100, frame_count)):
        ret, frame = vid.read()
        if ret and np.std(frame) < threshold:
            empty_frames += 1
    vid.release()
    return empty_frames / min(100, frame_count) > 0.5

df_filtered = []
for i, row in df.iterrows():
    if not check_empty_frames(row["video_path"]):
        df_filtered.append(row)
df = pd.DataFrame(df_filtered)
df.to_csv(os.path.join(OUTPUT_DIR, "labels_final.csv"), index=False)
print(f"Videos after removing empty frames: {len(df)}")

Videos after removing empty frames: 0


In [31]:
processor = VideoMAEImageProcessor.from_pretrained(MODEL_NAME)

class VideoMAEDataset(Dataset):
    def __init__(self, df, num_frames=16):
        self.df = df.reset_index(drop=True)
        self.num_frames = num_frames

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        video_path = self.df.iloc[idx]["video_path"]
        label = self.df.iloc[idx]["label"]

        vr = VideoReader(video_path, ctx=cpu(0))
        total_frames = len(vr)
        indices = np.linspace(0, total_frames - 1, self.num_frames, dtype=int)
        frames = vr.get_batch(indices).asnumpy()  # (T, H, W, C)

        inputs = processor(list(frames), return_tensors="pt")
        pixel_values = inputs["pixel_values"].squeeze(0)
        return pixel_values, torch.tensor(label)

NameError: name 'MODEL_NAME' is not defined

In [32]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_dataset = VideoMAEDataset(train_df)
val_dataset = VideoMAEDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

KeyError: 'label'

In [ ]:
from transformers import VideoMAEForVideoClassification, VideoMAEImageProcessor

# Pretrained model from Hugging Face
model_name = "MCG-NJU/videomae-base"
processor = VideoMAEImageProcessor.from_pretrained(model_name)
model = VideoMAEForVideoClassification.from_pretrained(
    model_name,
    num_labels=2  # shoplifter / non-shoplifter
)
model = model.to(DEVICE)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    for pixel_values, labels in train_loader:
        pixel_values, labels = pixel_values.to(DEVICE), labels.to(DEVICE)
        outputs = model(pixel_values)
        loss = criterion(outputs.logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
model.eval()
all_labels, all_probs = [], []

with torch.no_grad():
    for X, y in val_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        outputs = model(pixel_values=X)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)[:, 1]
        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

all_labels = np.array(all_labels)
all_probs = np.array(all_probs)
all_preds = (all_probs >= 0.5).astype(int)

cm = confusion_matrix(all_labels, all_preds)
acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
print("Confusion Matrix:\n", cm)
print(f"Accuracy: {acc:.4f}, F1-score: {f1:.4f}")

In [ ]:
!nvidia-smi
!nvcc --version


In [ ]:
fpr, tpr, _ = roc_curve(all_labels, all_probs)
roc_auc = roc_auc_score(all_labels, all_probs)
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f"AUC = {roc_auc:.4f}")
plt.plot([0,1],[0,1],'--', color='gray')
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve"); plt.legend(); plt.grid(True); plt.show()

precision, recall, _ = precision_recall_curve(all_labels, all_probs)
pr_auc = auc(recall, precision)
plt.figure(figsize=(6,6))
plt.plot(recall, precision, color='green', lw=2, label=f"PR AUC = {pr_auc:.4f}")
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title("Precision-Recall Curve"); plt.legend(); plt.grid(True); plt.show()